In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch

from collections import defaultdict
import os.path as osp
from typing import Union, Dict, List

filenames=["camelyon", "camelyon_np", "resnet_camelyon", "resnet_np_camelyon"]
res_path = "../logs/"

res = defaultdict(list)

print(f"{'METHOD':<30}\tTEST_ACC        TEST_ACC_STD")

for filename in filenames:

    filename_complete = osp.join(res_path, f"{filename}_results.json")

    with open(filename_complete) as f:
        logs=json.load(f)
        test_acc = logs["test"]["acc_avg"]
        test_acc_std = logs["test"]["acc_avg_std"]
        print(f"{filename:<30}\t{test_acc:.3f}              {test_acc_std:.3f}")


METHOD                        	TEST_ACC        TEST_ACC_STD
camelyon                      	0.896              0.020
camelyon_np                   	0.866              0.012
resnet_camelyon               	0.919              0.013
resnet_np_camelyon            	0.800              0.138


In [5]:
import torch
import torch
import torch.nn as nn
import numpy as np
import sys
sys.path.insert(0, '/datasets/home/hbenoit/DivDis-exp/wilds/')
from algorithms.DivDis import DivDis, MultiHeadModel
from models.initializer import initialize_torchvision_model
from utils import load, move_to
import wilds
from wilds.common.data_loaders import get_eval_loader, get_train_loader
from transforms import initialize_transform
import json
from argparse import Namespace
import argparse
import os

from wilds.common.grouper import CombinatorialGrouper

path = "/datasets/home/hbenoit/DivDis-exp/wilds/logs/camelyon_seed2/camelyon17_seed:2_epoch:best_model.pth"


res = torch.load(path)



In [40]:
class MultiHeadModel(nn.Module):
    def __init__(self, featurizer, classifier, heads=2):
        super().__init__()
        self.heads = heads
        self.featurizer = featurizer
        in_dim, out_dim = classifier.in_features, classifier.out_features * self.heads
        self.heads_classifier = nn.Linear(in_dim, out_dim)

    def forward(self, x):
        features = self.featurizer(x)
        outputs = self.heads_classifier(features)
        return outputs
    

    def process_batch(self, batch):
        """
        Overrides single_model_algorithm.process_batch().
        Args:
            - batch (x, y, m): a batch of data yielded by data loaders
        Output:
            - results (dictionary): information about the batch
                - y_true (Tensor): ground truth labels for batch

                - y_pred (Tensor): model output for batch

        """
        # Labeled examples
        x, y_true, metadata = batch
        x = move_to(x, DEVICE)
        y_true = move_to(y_true, DEVICE)
        # package the results
        results = { "y_true": y_true, "metadata": metadata}

        pred = self.forward(x)
        preds_chunked = torch.chunk(pred, self.heads, dim=-1)
        for i in range(self.heads):
            results[f"y_pred_{i}"] = preds_chunked[i]

        return results


featurizer = initialize_torchvision_model(
    name="densenet121", d_out=None, **{"pretrained":False}
)
classifier = nn.Linear(featurizer.d_out, 4)

import copy

model= res["algorithm"]

new_model = copy.deepcopy(model)
for key in model:
    if "model.featurizer." in key:
        new_key = key.replace("model.featurizer." ,"")
        new_model[new_key] = model[key]
        del new_model[key]
    elif "model.heads_classifier." in key:
        new_key = key.replace("model.heads_classifier.","")
        new_model[new_key] = model[key]
        del new_model[key]

featurizer.load_state_dict(new_model,strict=False)
classifier.load_state_dict({"weight":new_model["weight"], "bias":new_model["bias"]}, strict=True)

algorithm = MultiHeadModel(featurizer=featurizer, classifier=classifier)

RuntimeError: Error(s) in loading state_dict for MultiHeadModel:
	Missing key(s) in state_dict: "featurizer.features.conv0.weight", "featurizer.features.norm0.weight". 
	Unexpected key(s) in state_dict: "features.conv0.weight". 

In [23]:
new_model.keys()

odict_keys(['features.conv0.weight', 'features.norm0.bias', 'features.norm0.running_mean', 'features.norm0.running_var', 'features.norm0.num_batches_tracked', 'features.denseblock1.denselayer1.norm1.weight', 'features.denseblock1.denselayer1.norm1.bias', 'features.denseblock1.denselayer1.norm1.running_mean', 'features.denseblock1.denselayer1.norm1.running_var', 'features.denseblock1.denselayer1.norm1.num_batches_tracked', 'features.denseblock1.denselayer1.conv1.weight', 'features.denseblock1.denselayer1.norm2.weight', 'features.denseblock1.denselayer1.norm2.bias', 'features.denseblock1.denselayer1.norm2.running_mean', 'features.denseblock1.denselayer1.norm2.running_var', 'features.denseblock1.denselayer1.norm2.num_batches_tracked', 'features.denseblock1.denselayer1.conv2.weight', 'features.denseblock1.denselayer2.norm1.weight', 'features.denseblock1.denselayer2.norm1.bias', 'features.denseblock1.denselayer2.norm1.running_mean', 'features.denseblock1.denselayer2.norm1.running_var', 'fea